# Problema de los filosofos

Cinco filósofos dedican sus vidas a pensar y comer. Los filósofos comparten una mesa
circular rodeada por cinco sillas, cada una de las cuáles pertenece a un filósofo. En la
mesa hay cinco platos de arroz y cinco palillos.

In [ ]:
import threading
import time

class Palillo:
    def __init__(self,numero,cerrojo):
        self.numero=numero
        self.cerrojo=cerrojo
        
    def obtenerPalillo(self):
        self.cerrojo.acquire()
        print("El palillo "+str(self.numero)+" lo tiene el filosofo "+threading.currentThread().getName()+"\n")
            
    def soltarPalillo(self):
        #print("El filosofo "+threading.currentThread().getName()+" suelta el palillo "+str(self.numero)+"\n")
        self.cerrojo.release()
        
    
        

In [ ]:
class Filosofo(threading.Thread):
    
    def __init__(self,nombre, id, izdo,dcho):
        threading.Thread.__init__(self)
        self.name=nombre
        self.id=id
        self.izdo=izdo
        self.dcho=dcho

        
    def run(self):
    
        for i in range(2):
            print("El filosofo "+threading.currentThread().getName()+" ha entrado a comer\n ")
            self.izdo.obtenerPalillo()
            time.sleep(0.001)
            self.dcho.obtenerPalillo()
            print("El filosofo "+threading.currentThread().getName()+" está comiendo\n ")
            self.izdo.soltarPalillo()
            self.dcho.soltarPalillo()
            print("El filosofo "+threading.currentThread().getName()+" está pensando\n ")
           
            
            

        

In [ ]:
if __name__=="__main__":
    palillos=[]
    
    for i  in range(5):
        cerrojo=threading.Lock() 
        palillo=Palillo(i,cerrojo)
        palillos.append(palillo)
        
    filosofos=[]
    for i in range(5):
        nombre="Filosofo "+str(i)
        filosofo=Filosofo(nombre,i,palillos[i],palillos[(i+1)%5])
        filosofos.append(filosofo)
       
    for i in range(5):
        filosofos[i].start()
        
    for i in range(5):
        filosofos[i].join()
       
        


   
        
        
    

A continuación se presenta una nueva solución. En este caso, si el filosofo después de coger el primer palillo no puede coger el segundo, libera el primero de ellos.
Esta solución, sin embargo, puede que produzca un problema de inanición en alguno de los filósofos si nunca puede llegar a coger los dos.

In [2]:
import threading
import time

class PalilloConEstado:
    def __init__(self,numero):
        self.numero=numero
        self.cerrojo=threading.Lock()
        self.ocupado=False # se necesita para poder comprobar previamente el estado del segundo palillo
        
    def obtenerPalillo(self):
        self.cerrojo.acquire()
        self.ocupado=True
        print("El palillo "+str(self.numero)+" lo tiene el filosofo "+threading.currentThread().getName()+"\n")
            
    def soltarPalillo(self):
        print("El filosofo "+threading.currentThread().getName()+" suelta el palillo "+str(self.numero)+"\n")
        self.ocupado=False
        self.cerrojo.release()
        
class FilosofoConLiberacion(threading.Thread):
    
    def __init__(self,nombre, id, izdo,dcho):
        threading.Thread.__init__(self)
        self.name=nombre
        self.id=id
        self.izdo=izdo
        self.dcho=dcho

        
    def run(self):
    
        for i in range(2):
            print("El filosofo "+threading.currentThread().getName()+" ha entrado a comer\n ")
            comer=False
            while not comer:
                self.izdo.obtenerPalillo()
                if self.dcho.ocupado:
                    self.izdo.soltarPalillo()
                    time.sleep(2)
                else:
                    self.dcho.obtenerPalillo()
                    comer=True
            
            print("El filosofo "+threading.currentThread().getName()+" está comiendo\n ")
            time.sleep(2)
            self.izdo.soltarPalillo()
            self.dcho.soltarPalillo()
            print("El filosofo "+threading.currentThread().getName()+" está pensando\n ")
            time.sleep(2)
if __name__=="__main__":
    palillosConEstado=[]
    
    for i  in range(5):
        palillo=PalilloConEstado(i)
        palillosConEstado.append(palillo)
        
    filosofosCL=[]
    for i in range(5):
        nombre="Filosofo "+str(i)
        filosofo=FilosofoConLiberacion(nombre,i,palillosConEstado[i],palillosConEstado[(i+1)%5])
        filosofosCL.append(filosofo)
       
    for i in range(5):
        filosofosCL[i].start()
        
    for i in range(5):
        filosofosCL[i].join()
            

El filosofo Filosofo 0 ha entrado a comer
 
El palillo 0 lo tiene el filosofo Filosofo 0

El palillo 1 lo tiene el filosofo Filosofo 0

El filosofo Filosofo 0 está comiendo
 
El filosofo Filosofo 1 ha entrado a comer
 
El filosofo Filosofo 2 ha entrado a comer
 
El palillo 2 lo tiene el filosofo Filosofo 2

El palillo 3 lo tiene el filosofo Filosofo 2

El filosofo Filosofo 2 está comiendo
 
El filosofo Filosofo 3 ha entrado a comer
 El filosofo Filosofo 4 ha entrado a comer
 

El palillo 4 lo tiene el filosofo Filosofo 4

El filosofo Filosofo 4 suelta el palillo 4

El filosofo Filosofo 0 suelta el palillo 0
El filosofo Filosofo 2 suelta el palillo 2

El filosofo Filosofo 2 suelta el palillo 3

El filosofo Filosofo 2 está pensando
 
El filosofo Filosofo 0 suelta el palillo 1

El palillo 3 lo tiene el filosofo Filosofo 3

El filosofo Filosofo 0 está pensando
 
El palillo 1 lo tiene el filosofo Filosofo 1

El palillo 2 lo tiene el filosofo Filosofo 1

El filosofo Filosofo 1 está comiendo


La última solución consiste en limitar el acceso a la sala a más de cuatro filósofos. Para esto se va a usar un semáforo inicializado a 4 que controla el acceso al comedor como si hubiese un camarero.

In [5]:
import threading
import time

camarero=threading.Semaphore(4)

class PalilloCamarero:
    def __init__(self,numero):
        self.numero=numero
        self.cerrojo=threading.Lock()
        self.ocupado=False # se necesita para poder comprobar previamente el estado del segundo palillo
        
    def obtenerPalillo(self):
        self.cerrojo.acquire()
        self.ocupado=True
        print("El palillo "+str(self.numero)+" lo tiene el filosofo "+threading.currentThread().getName()+"\n")
            
    def soltarPalillo(self):
        print("El filosofo "+threading.currentThread().getName()+" suelta el palillo "+str(self.numero)+"\n")
        self.ocupado=False
        self.cerrojo.release()
        
class FilosofoCamarero(threading.Thread):
    
    def __init__(self,nombre, id, izdo,dcho):
        threading.Thread.__init__(self)
        self.name=nombre
        self.id=id
        self.izdo=izdo
        self.dcho=dcho

        
    def run(self):
    
        for i in range(2):
            camarero.acquire()
            print("El filosofo "+threading.currentThread().getName()+" ha entrado a comer\n ")
            self.izdo.obtenerPalillo()
            self.dcho.obtenerPalillo()
            
            print("El filosofo "+threading.currentThread().getName()+" está comiendo\n ")
            time.sleep(2)
            self.izdo.soltarPalillo()
            self.dcho.soltarPalillo()
            camarero.release()
            print("El filosofo "+threading.currentThread().getName()+" está pensando\n ")
            time.sleep(2)
if __name__=="__main__":
     #limita el acceso a 4 filósofos
    palillosCamarero=[]
    
    for i  in range(5):
        palillo=PalilloCamarero(i)
        palillosCamarero.append(palillo)
        
    filosofosCamarero=[]
    for i in range(5):
        nombre="Filosofo "+str(i)
        filosofo=FilosofoCamarero(nombre,i,palillosConEstado[i],palillosConEstado[(i+1)%5])
        filosofosCamarero.append(filosofo)
       
    for i in range(5):
        filosofosCamarero[i].start()
        
    for i in range(5):
        filosofosCamarero[i].join()
            

El filosofo Filosofo 0 ha entrado a comer
 
El palillo 0 lo tiene el filosofo Filosofo 0

El palillo 1 lo tiene el filosofo Filosofo 0

El filosofo Filosofo 0 está comiendo
 
El filosofo Filosofo 1 ha entrado a comer
 
El filosofo Filosofo 2 ha entrado a comer
 
El palillo 2 lo tiene el filosofo Filosofo 2

El palillo 3 lo tiene el filosofo Filosofo 2
El filosofo Filosofo 3 ha entrado a comer
 
El filosofo Filosofo 2 está comiendo
 

El filosofo Filosofo 0 suelta el palillo 0

El filosofo Filosofo 0 suelta el palillo 1

El filosofo Filosofo 0 está pensando
 
El palillo 1 lo tiene el filosofo Filosofo 1

El filosofo Filosofo 4 ha entrado a comer
 
El palillo 4 lo tiene el filosofo Filosofo 4

El palillo 0 lo tiene el filosofo Filosofo 4

El filosofo Filosofo 4 está comiendo
 
El filosofo Filosofo 2 suelta el palillo 2

El filosofo Filosofo 2 suelta el palillo 3

El filosofo Filosofo 2 está pensando
 
El palillo 2 lo tiene el filosofo Filosofo 1

El filosofo Filosofo 1 está comiendo
 
El